<a href="https://colab.research.google.com/github/jrenatom/tarefa1_PL_A1/blob/main/TarefaAnac2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math

import networkx as nx
import matplotlib.pyplot as plt

In [2]:
airport_col = ['ID', 'Name', 'City', 'Country','IATA', 'ICAO', 'Lat', 'Long', 'Alt', 
	               'Timezone', 'DST', 'Tz database time zone', 'type', 'source']
airport_df = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat",
	                        names = airport_col, index_col = 0)
airport_df[airport_df.Country == 'Brazil'].head(20)

,Name,City,Country,IATA,ICAO,Lat,Long,Alt,Timezone,DST,Tz database time zone,type,source
ID,,,,,,,,,,,,,
2518,Conceição do Araguaia Airport,Conceicao Do Araguaia,Brazil,CDJ,SBAA,-8.348350,-49.301498,653,-3,S,America/Belem,airport,OurAirports
2519,Campo Délio Jardim de Mattos Airport,Rio De Janeiro,Brazil,\N,SBAF,-22.875099,-43.384701,110,-3,S,America/Sao_Paulo,airport,OurAirports
2520,Amapá Airport,Amapa,Brazil,\N,SBAM,2.077510,-50.858200,45,-3,S,America/Fortaleza,airport,OurAirports
2521,Araraquara Airport,Araracuara,Brazil,AQA,SBAQ,-21.812000,-48.132999,2334,-3,S,America/Sao_Paulo,airport,OurAirports
2522,Santa Maria Airport,Aracaju,Brazil,AJU,SBAR,-10.984000,-37.070301,23,-3,S,America/Fortaleza,airport,OurAirports
2524,Piloto Osvaldo Marques Dias Airport,Alta Floresta,Brazil,AFL,SBAT,-9.866389,-56.105000,948,-4,S,America/Campo_Grande,airport,OurAirports
2525,Araçatuba Airport,Aracatuba,Brazil,ARU,SBAU,-21.141300,-50.424702,1361,-3,S,America/Sao_Paulo,airport,OurAirports
2526,Val de Cans/Júlio Cezar Ribeiro International ...,Belem,Brazil,BEL,SBBE,-1.379250,-48.476299,54,-3,S,America/Belem,airport,OurAirports
2527,Comandante Gustavo Kraemer Airport,Bage,Brazil,BGX,SBBG,-31.390499,-54.112202,600,-3,S,America/Sao_Paulo,airport,OurAirports


In [3]:
def monta_pos(IATA):
  airport_pos_df = airport_df.loc[airport_df['IATA'].isin(IATA)][['IATA', 'Lat', 'Long']]
  airport_pos_df.reset_index(inplace=True)
  
  pos = {}
  for index, elem in enumerate (airport_pos_df['IATA']):    
    pos[elem] =  (airport_pos_df.loc[index ,'Long'], airport_pos_df.loc[index ,'Lat']) 

  found = len(airport_pos_df.loc[~airport_pos_df['Lat'].isnull()])
  print(f'{found} Aeroportos com Lat, Long')  

  return pos

In [4]:
def routes_top(routes_grouped_df, top):
  routes_grouped_df_top = routes_grouped_df.loc[routes_grouped_df['counts'] > top]  

  IATA_dest = set(routes_grouped_df_top['IATA_dest'].unique())
  print(f'{len(IATA_dest)} Aeroportos encontrados destinos')

  IATA_orig = set(routes_grouped_df_top['IATA_orig'].unique())  
  print(f'{len(IATA_orig)} Aeroportos encontrados origem')

  IATA = IATA_dest.union(IATA_orig)
  print(f'{len(IATA)} Aeroportos encontrados total')
  return IATA, routes_grouped_df_top

In [5]:
def monta_rotas(top):
  ######################################################
  # Define nomes mais faceis de manipular para a tabela
  ######################################################
  '''
  0 'ICAO Empresa Aérea'	'Número Voo'	'Código Autorização (DI)'	'Código Tipo Linha'	'ICAO Aeródromo Origem'	'ICAO Aeródromoo Destino'	
  'Partida Prevista'	'Partida Real'	'Chegada Prevista'	'Chegada Real'	
  'Situação Voo'	'Código Justificativa'	'ano'	'mes'                 
  '''

  routes_col = ['company_id', 'flight_no', 'auth_code', 'line_code','ICAO_orig', 'ICAO_dest', 
                'depature_estimated', 'depature_time', 'arrival_estimated', 'arriaval_time', 
                'flight_status', 'flight_status_cause_code', 'ano', 'mes']

  #Carrega a lista do ano, porém eliminando os voos cancelados
  routes_df = pd.read_csv(f"https://github.com/lucianomcsilva/VRA/blob/main/VRA_ANO_2018.csv.zip?raw=true"
                            ,compression='zip' ,header=0 ,names = routes_col, index_col = 0) #

  routes_df = routes_df.loc[routes_df['flight_status'] != 'CANCELADO']

  #Agrupa, mantendo so o que entrara no grafo
  routes_grouped_df = routes_df.groupby(['ICAO_orig', 'ICAO_dest']).size().reset_index(name='counts')                          
  
  ######################################################################
  #Jaz a junção com o Airport.dat para trocar para o codigo de 3 letras
  ######################################################################
  #Join origem
  routes_grouped_df = routes_grouped_df.merge(airport_df[['ICAO', 'IATA', 'Lat', 'Long', 'Country']], how='left', left_on='ICAO_orig', right_on='ICAO')
  routes_grouped_df = routes_grouped_df[['IATA', 'Country', 'ICAO_dest', 'counts']]
  routes_grouped_df.rename(columns={'IATA': 'IATA_orig', 'Country': 'Country_orig'}, inplace = True)
  #Join Destino
  routes_grouped_df = routes_grouped_df.merge(airport_df[['ICAO', 'IATA', 'Lat', 'Long', 'Country']], how='left', left_on='ICAO_dest', right_on='ICAO')
  routes_grouped_df = routes_grouped_df[['IATA_orig', 'Country_orig', 'IATA', 'Country', 'counts']]
  routes_grouped_df.rename(columns={'IATA': 'IATA_dest', 'Country': 'Country_dest'}, inplace = True)

  #Filtra somente Brazil, e mantem somente as colunas necessarias
  routes_grouped_df = (routes_grouped_df
                        .loc[routes_grouped_df['Country_orig'] == 'Brazil']
                        .loc[routes_grouped_df['Country_dest'] == 'Brazil']
                        .loc[routes_grouped_df['IATA_orig'] != routes_grouped_df['IATA_dest']]
                        .loc[routes_grouped_df['IATA_orig'].str.len() == 3]
                        .loc[routes_grouped_df['IATA_dest'].str.len() == 3]
                        .sort_values(by=['counts'], ascending=False)
                      )
  routes_grouped_df = routes_grouped_df[['IATA_orig', 'IATA_dest', 'counts']]  
  
  #prepara os DataFrames que serão utilizados
  IATA, routes_grouped_df_top = routes_top(routes_grouped_df, top)
  pos = monta_pos(IATA)
  return routes_grouped_df_top, pos

  #lista de aeroportos nao encontrados
  #routes_grouped_df.loc[routes_grouped_df['ICAO'].isnull()]['ICAO_orig'].to_list()
  #df_anac_2015 = pd.read_csv(f"VRA_ANO_2015.csv")

In [6]:
routes, pos = monta_rotas(1)
routes.reset_index(inplace=True)
display(len(routes))
display(len(pos))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


117 Aeroportos encontrados destinos
116 Aeroportos encontrados origem
118 Aeroportos encontrados total
118 Aeroportos com Lat, Long


1090

118

In [ ]:
G = nx.Graph()
airport_brasil_df = airport_df.loc[airport_df['Country'] == 'Brazil'].loc[airport_df['IATA'].str.len() == 3]
airport_brasil_df.reset_index(inplace=True)

for index, elem in enumerate(routes['IATA_orig']):
  G.add_edge(routes.loc[index, 'IATA_orig'], routes.loc[index, 'IATA_dest'])

#Dois aeroportos desconexos. Nao faz sentido remover caso desenhe todos os aeroportos, pois haverão outros disconexos (sem voo algum, valencia = 0)
G.remove_node('SXO')
G.remove_node('CFO')

options = {
    "font_size": 18,
    "node_size": 100,
    "node_color": "#336699CC",
    "edgecolors": "#002060",
    "edge_color": "#D34817",
    "linewidths": 5,
    "width": [1+int(math.log(x, 30)) for x in routes['counts'].to_list()],
    "with_labels": True
}

plt.figure(figsize = (15,15))
nx.draw_networkx(G, pos, **options)
#plt.savefig(f'brasil_2_map_{ano}.png', dpi=50)

In [8]:
print(f"O diametro dessa rede  é {nx.diameter(G)}")
print(f"O coeficiente médio de clustering dessa rede é {nx.average_clustering(G)}")
print(f"A distancia média entre os nodos é de  {nx.average_shortest_path_length(G):.2f}")
print(nx.clustering(G))
#print(nx.triads.all_triplets(G))

O diametro dessa rede  é 5
O coeficiente médio de clustering dessa rede é 0.5963255265721131
A distancia média entre os nodos é de  2.35
{'CGH': 0.41585365853658535, 'SDU': 0.5928853754940712, 'GRU': 0.3018867924528302, 'POA': 0.34992887624466573, 'BSB': 0.3200354609929078, 'CNF': 0.2585858585858586, 'CWB': 0.6123076923076923, 'SSA': 0.4111111111111111, 'REC': 0.4717741935483871, 'FOR': 0.5846153846153846, 'GIG': 0.4076655052264808, 'FLN': 0.6405228758169934, 'VCP': 0.19764812302125734, 'GYN': 0.6719367588932806, 'NVT': 0.8974358974358975, 'VIX': 0.6405228758169934, 'IGU': 0.6758893280632411, 'NAT': 0.8014705882352942, 'CGB': 0.3277310924369748, 'MCZ': 0.8210526315789474, 'MAO': 0.21149425287356322, 'UDI': 0.8161764705882353, 'BEL': 0.5105263157894737, 'CGR': 0.6544117647058824, 'BPS': 0.6096866096866097, 'LDB': 0.738562091503268, 'MCP': 1.0, 'FEN': 1.0, 'MOC': 0.7142857142857143, 'JOI': 0.7878787878787878, 'SJP': 0.8363636363636363, 'RAO': 0.6268115942028986, 'AJU': 0.8974358974358975